In [2]:
from pyspark.sql import SparkSession
import pyspark .sql.functions as F

In [ ]:
spark = (
    SparkSession.builder.appName('Desafio PySpark - Programador Aventureiro')
    .config('spark.sql.repl.eagerEval.enabled', True)
    .getOrCreate()
)

# Carregando o DataFrame "Titanic"

In [31]:
titanic = spark.read.csv('./titanic/train.csv', sep=',', encoding='UTF-8', header=True, inferSchema=True)
titanic

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|          6|       0|     3|    Moran, Mr. James|  male|NULL|    0|    0|          330877| 8.4583| NULL|       Q|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|           17463|51.8625|  E46|       S|
|          8|       0|     3|Palsson, Master. ...|  male| 2.0|    3|    1|          349909| 21.075| NULL|       S|
|          9|       1|     3|Johnson, Mrs. Osc...|female|27.0|    0|    2|          347742|11.1333| NULL|       S|
|         10|       1|     2|Nasser, Mrs. Nich...|female|14.0|    1|    0|          237736|30.0708| NULL|       C|
|         11|       1|     3|Sandstrom, Miss. ...|female| 4.0|    1|    1|         PP 9549|   16.7|   G6|       S|
|         12|       1|     1|Bonnell, Miss. El...|female|58.0|    0|    0|          113783|  26.55| C103|       S|
|         13|       0|     3|Saundercock, Mr. ...|  male|20.0|    0|    0|       A/5. 2151|   8.05| NULL|       S|
|         14|       0|     3|Andersson, Mr. An...|  male|39.0|    1|    5|          347082| 31.275| NULL|       S|
|         15|       0|     3|Vestrom, Miss. Hu...|female|14.0|    0|    0|          350406| 7.8542| NULL|       S|
|         16|       1|     2|Hewlett, Mrs. (Ma...|female|55.0|    0|    0|          248706|   16.0| NULL|       S|
|         17|       0|     3|Rice, Master. Eugene|  male| 2.0|    4|    1|          382652| 29.125| NULL|       Q|
|         18|       1|     2|Williams, Mr. Cha...|  male|NULL|    0|    0|          244373|   13.0| NULL|       S|
|         19|       0|     3|Vander Planke, Mr...|female|31.0|    1|    0|          345763|   18.0| NULL|       S|
|         20|       1|     3|Masselmani, Mrs. ...|female|NULL|    0|    0|            2649|  7.225| NULL|       C|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 20 rows

---
# Pergunta 1

### Qual a proporção de sobreviventes em relação ao total de passageiros a bordo do Titanic?

In [28]:
perc_sobreviventes = titanic.where(F.col('Survived') == 1).count() / titanic.count()
print(f"O percentual de sobreviventes é: {perc_sobreviventes:.2%}")

O percentual de sobreviventes é: 38.38%


---
# Pergunta 2

### Qual a média de idade dos passageiros que não sobreviveram?

In [33]:
# Podemos usar o `describe` para pegar os dados estatísticos da coluna `Age` após filtrar os que não sobreviveram
titanic.where(F.col('Survived') == 0).describe('Age')

+-------+------------------+
|summary|               Age|
+-------+------------------+
|  count|               424|
|   mean| 30.62617924528302|
| stddev|14.172109877138494|
|    min|               1.0|
|    max|              74.0|
+-------+------------------+

In [35]:
# Ou podemos calcular a média da coluna `Age` após filtrar os que não sobreviveram
media_idade = titanic.where(F.col('Survived') == 0).agg(F.mean('Age'))

In [50]:
print(f"A média de idade dos passageiros que não sobreviveram é de {int(media_idade.first()[0])} anos")

A média de idade dos passageiros que não sobreviveram é de 30 anos


---
# Pergunta 3

### Quantos passageiros da terceira classe sobreviveram ao naufrágio?

In [53]:
sobreviventes_terceira_classe = titanic.where(F.col('Survived') == 1).where(F.col('Pclass') == 3).count()
print(f"Na terceira classe sobreviveram {sobreviventes_terceira_classe} passageiros")

Na terceira classe sobreviveram 119 passageiros


---
# Pergunta 4

### Qual a porcentagem de mulheres sobreviventes em relação ao total de mulheres a bordo do Titanic?

In [105]:
mulheres = titanic.where(F.col('Sex') == 'female')
perc_mulheres_sobreviventes = mulheres.where(F.col('Survived') == 1).count() / mulheres.count()
print(f"Sobreviveram {perc_mulheres_sobreviventes:.1%} das mulheres a bordo")

Sobreviveram 74.2% das mulheres a bordo


---
# Pergunta 5

### Qual foi o preço médio da passagem dos passageiros que sobreviveram ao naufrágio?

In [64]:
tarifa_media_sobreviventes = titanic.where(F.col('Survived') == 1).agg(F.mean('Fare'))
print(f"O preço médio pago pelos sobreviventes foi de ${tarifa_media_sobreviventes.first()[0]:.2f}")

O preço médio pago pelos sobreviventes foi de $48.40


---
# Pergunta 6

### Qual a quantidade de passageiros que embarcaram no Titanic?

In [65]:
total = titanic.count()
print(f"No total embarcaram {total} passageiros no Titanic")

No total embarcaram 891 passageiros no Titanic


---
# Pergunta 7

### Qual a proporção de passageiros do sexo masculino em relação ao total de passageiros?

In [66]:
homens = titanic.where(F.col('Sex') == 'male')
perc_homens = homens.count() / titanic.count()
print(f"Do total de passageiros {perc_homens:.0%} eram do sexo masculino")

Do total de passageiros 65% eram do sexo masculino


---
# Pergunta 8

### Qual a proporção de passageiros da terceira classe que sobreviveram?

In [116]:
terceira_classe = titanic.where(F.col('Pclass') == 3)
perc_sobreviventes = terceira_classe.where(F.col('Survived') == 1).count() / terceira_classe.count()
print(f"Dos passageiros da terceiras classe {perc_sobreviventes:.0%} deles sobreviveram")

Dos passageiros da terceiras classe 24% deles sobreviveram


---
---
---

# Dataset "Mall Customer Segmentation Data"

In [92]:
mall = spark.read.csv('./Mall_Customers.csv', sep=',', encoding='UTF-8', header=True, inferSchema=True)
mall

+----------+------+---+------------------+----------------------+
|CustomerID|Gender|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         2|  Male| 21|                15|                    81|
|         3|Female| 20|                16|                     6|
|         4|Female| 23|                16|                    77|
|         5|Female| 31|                17|                    40|
|         6|Female| 22|                17|                    76|
|         7|Female| 35|                18|                     6|
|         8|Female| 23|                18|                    94|
|         9|  Male| 64|                19|                     3|
|        10|Female| 30|                19|                    72|
|        11|  Male| 67|                19|                    14|
|        12|Female| 35|                19|                    99|
|        13|Female| 58|                20|                    15|
|        14|Female| 24|                20|                    77|
|        15|  Male| 37|                20|                    13|
|        16|  Male| 22|                20|                    79|
|        17|Female| 35|                21|                    35|
|        18|  Male| 20|                21|                    66|
|        19|  Male| 52|                23|                    29|
|        20|Female| 35|                23|                    98|
+----------+------+---+------------------+----------------------+
only showing top 20 rows

---
# Pergunta 9
### Qual é a média de idade dos clientes no Shopping?

In [84]:
mall.describe('Age')

+-------+-----------------+
|summary|              Age|
+-------+-----------------+
|  count|              200|
|   mean|            38.85|
| stddev|13.96900733155888|
|    min|               18|
|    max|               70|
+-------+-----------------+

In [86]:
idade_media = mall.agg(F.mean('Age'))
print(f"A idade média dos clientes é {idade_media.first()[0]:.2f} anos")

A idade média dos clientes é 38.85 anos


---
# Pergunta 10
### Quantas mulheres são clientes do Shopping?

In [91]:
clientes_mulheres = mall.where(F.col('Gender') == 'Female').count()
print(f"O shopping tem {clientes_mulheres} clientes do sexo feminino")

O shopping tem 112 clientes do sexo feminino


---
# Pergunta 11
### Qual é a média de renda anual dos clientes?

In [94]:
mall.describe('Annual Income (k$)')

+-------+------------------+
|summary|Annual Income (k$)|
+-------+------------------+
|  count|               200|
|   mean|             60.56|
| stddev| 26.26472116527124|
|    min|                15|
|    max|               137|
+-------+------------------+

In [96]:
renda_media_anual = mall.agg(F.mean('Annual Income (k$)'))
print(f"A média de renda anual dos clientes do shopping é de ${renda_media_anual.first()[0]:.2f} mil dólares")

A média de renda anual dos clientes do shopping é de $60.56 mil dólares


---
# Pergunta 12
### Qual é o desvio padrão da idade dos clientes?

In [97]:
mall.describe('Age')

+-------+-----------------+
|summary|              Age|
+-------+-----------------+
|  count|              200|
|   mean|            38.85|
| stddev|13.96900733155888|
|    min|               18|
|    max|               70|
+-------+-----------------+

In [99]:
desv_idade = mall.agg(F.stddev('Age'))
print(f"O desvio padrão da idade dos clientes é {desv_idade.first()[0]:.2f}")

O desvio padrão da idade dos clientes é 13.97


---
# Pergunta 13
### Quantos clientes têm uma renda anual superior a 80 mil?

In [117]:
renda_acima_80k = mall.where(F.col('Annual Income (k$)') > 80).count()
print(f"No total são {renda_acima_80k} clientes com renda acima de $80 mil")

No total são 38 clientes com renda acima de $80 mil


---
# Pergunta 14
### Qual é a moda da pontuação de gastos dos clientes?

In [104]:
moda = mall.agg(F.mode('Spending Score (1-100)'))
print(f"A pontuação mais frequente (moda) é {moda.first()[0]} pontos")

A pontuação mais frequente (moda) é 42 pontos
